In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.decomposition import PCA

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
df = pd.read_csv('gurgaon_properties_post_feature_selection1.csv')

In [5]:
df.head()

,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 112,4.75,3,5,3+,Relatively New,3405.0,1,0,0,Low,Mid Floor
1,flat,sector 37c,0.90,3,3,3,Under Construction,1500.0,0,0,1,Medium,Low Floor
2,flat,sector 81,0.45,2,2,2,Relatively New,664.0,0,1,0,Low,Mid Floor
3,house,sector 43,9.85,5,5,2,Old Property,3323.0,1,0,1,Low,Mid Floor
4,flat,sector 92,0.80,4,4,3,Moderately Old,1893.0,0,0,1,Medium,Low Floor


In [6]:
df['furnishing_type'].value_counts()

furnishing_type
1    2392
0     995
2     184
Name: count, dtype: int64

In [7]:
# 0 -> unfurnished
# 1 -> semifurnished
# 2 -> furnished
df['furnishing_type'] = df['furnishing_type'].replace({0.0:'unfurnished',1.0:'semifurnished',2.0:'furnished'})

In [8]:
df.head()

,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 112,4.75,3,5,3+,Relatively New,3405.0,1,0,unfurnished,Low,Mid Floor
1,flat,sector 37c,0.90,3,3,3,Under Construction,1500.0,0,0,semifurnished,Medium,Low Floor
2,flat,sector 81,0.45,2,2,2,Relatively New,664.0,0,1,unfurnished,Low,Mid Floor
3,house,sector 43,9.85,5,5,2,Old Property,3323.0,1,0,semifurnished,Low,Mid Floor
4,flat,sector 92,0.80,4,4,3,Moderately Old,1893.0,0,0,semifurnished,Medium,Low Floor


In [9]:
X = df.drop(columns=['price'])
y = df['price']

In [10]:
# Applying the log1p transformation to the target variable
y_transformed = np.log1p(y)

### Ordinal Encoding

In [11]:
columns_to_encode = ['property_type','sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

In [12]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1), columns_to_encode)
    ], 
    remainder='passthrough'
)

In [13]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [14]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [15]:
scores.mean(),scores.std()

(np.float64(0.6127622969350035), np.float64(0.0326402962366933))

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

In [17]:
pipeline.fit(X_train,y_train)

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [18]:
y_pred = pipeline.predict(X_test)

In [19]:
y_pred = np.expm1(y_pred)

In [20]:
mean_absolute_error(np.expm1(y_test),y_pred)

0.9376272402540762

In [21]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output
    

In [24]:
pip install xgboost

   ---------------------------------------- 0.0/56.8 MB ? eta -:--:--
   - -------------------------------------- 2.1/56.8 MB 11.7 MB/s eta 0:00:05
   --- ------------------------------------ 4.7/56.8 MB 11.4 MB/s eta 0:00:05
   ---- ----------------------------------- 7.1/56.8 MB 11.2 MB/s eta 0:00:05
   ------ --------------------------------- 9.4/56.8 MB 11.5 MB/s eta 0:00:05
   -------- ------------------------------- 11.8/56.8 MB 11.4 MB/s eta 0:00:04
   --------- ------------------------------ 14.2/56.8 MB 11.4 MB/s eta 0:00:04
   ----------- ---------------------------- 16.8/56.8 MB 11.5 MB/s eta 0:00:04
   ------------- -------------------------- 18.9/56.8 MB 11.2 MB/s eta 0:00:04
   -------------- ------------------------- 21.2/56.8 MB 11.4 MB/s eta 0:00:04
   ---------------- ----------------------- 23.6/56.8 MB 11.4 MB/s eta 0:00:03
   ------------------ --------------------- 26.2/56.8 MB 11.5 MB/s eta 0:00:03
   -------------------- ------------------- 28.6/56.8 MB 11.5 MB/

In [25]:
from sklearn.linear_model import Ridge , Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
import xgboost
from xgboost import XGBRegressor

In [26]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [27]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

In [28]:
model_output

[['linear_reg', np.float64(0.6127622969350035), 0.9376272402540762],
 ['svr', np.float64(0.6687630119018878), 0.8731022490720901],
 ['ridge', np.float64(0.6127667728801636), 0.9377548137480418],
 ['LASSO', np.float64(0.053503608389867806), 1.4205735075392303],
 ['decision tree', np.float64(0.7773656547886747), 0.7128707327027058],
 ['random forest', np.float64(0.8811798942939021), 0.4880684424667241],
 ['extra trees', np.float64(0.8676096644931782), 0.5068223179390178],
 ['gradient boosting', np.float64(0.8753648519428774), 0.5265456688754344],
 ['adaboost', np.float64(0.7450722120811302), 0.76243919549692],
 ['mlp', np.float64(0.7942487864349286), 0.684596807839159],
 ['xgboost', np.float64(0.8885585406980004), 0.4937674977545971]]

In [29]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [30]:
model_df.sort_values(['mae'])

,name,r2,mae
5,random forest,0.881180,0.488068
10,xgboost,0.888559,0.493767
6,extra trees,0.867610,0.506822
7,gradient boosting,0.875365,0.526546
9,mlp,0.794249,0.684597
4,decision tree,0.777366,0.712871
8,adaboost,0.745072,0.762439
1,svr,0.668763,0.873102
0,linear_reg,0.612762,0.937627
2,ridge,0.612767,0.937755


### OneHotEncoding

In [36]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',handle_unknown='ignore'),['sector','agePossession','furnishing_type'])
    ], 
    remainder='passthrough'
)

In [37]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [38]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [39]:
scores.mean()

np.float64(0.7878951262412041)

In [40]:
scores.std()

np.float64(0.0322813507100919)

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)

In [42]:
pipeline.fit(X_train,y_train)

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [43]:
y_pred = pipeline.predict(X_test)

In [44]:
y_pred = np.expm1(y_pred)

In [45]:
mean_absolute_error(np.expm1(y_test),y_pred)

0.6968115462137721

In [46]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output
    

In [49]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [50]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

In [51]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [52]:
model_df.sort_values(['mae'])

,name,r2,mae
6,extra trees,0.890196,0.441289
10,xgboost,0.893063,0.453715
5,random forest,0.891243,0.471683
7,gradient boosting,0.875552,0.523600
9,mlp,0.866386,0.535245
4,decision tree,0.812028,0.645699
2,ridge,0.788781,0.694733
0,linear_reg,0.787895,0.696812
8,adaboost,0.746795,0.779088
1,svr,0.672029,0.869583


### OneHotEncoding With PCA

In [68]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',handle_unknown='ignore',sparse_output=False),['sector','agePossession'])
    ], 
    remainder='passthrough'
)

In [69]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('pca', PCA(n_components=0.95)),
    ('regressor', LinearRegression())
])

In [70]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [71]:
scores.mean()

np.float64(0.05584685254920585)

In [72]:
scores.std()

np.float64(0.021535266637357085)

In [73]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('pca', PCA(n_components=0.95)),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output
    

In [74]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [75]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

In [76]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [77]:
model_df.sort_values(['mae'])

,name,r2,mae
5,random forest,0.698363,0.813482
6,extra trees,0.655977,0.848001
4,decision tree,0.595174,0.935932
10,xgboost,0.598598,0.951211
7,gradient boosting,0.590251,1.005219
1,svr,0.225911,1.239075
8,adaboost,0.282953,1.244489
9,mlp,0.190001,1.311464
3,LASSO,0.053646,1.420528
2,ridge,0.055847,1.427571


### Target Encoder

In [80]:
!pip install category_encoders

In [81]:
import category_encoders as ce

columns_to_encode = ['property_type','sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',handle_unknown='ignore',sparse_output=False),['agePossession']),
        ('target_enc', ce.TargetEncoder(), ['sector'])
    ], 
    remainder='passthrough'
)

In [82]:
# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [83]:
# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')

In [84]:
scores.mean(),scores.std()

(np.float64(0.7527538965430034), np.float64(0.027449141711282866))

In [85]:
def scorer(model_name, model):
    
    output = []
    
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    
    # K-fold cross-validation
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)
    
    pipeline.fit(X_train,y_train)
    
    y_pred = pipeline.predict(X_test)
    
    y_pred = np.expm1(y_pred)
    
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))
    
    return output
    

In [86]:
model_dict = {
    'linear_reg':LinearRegression(),
    'svr':SVR(),
    'ridge':Ridge(),
    'LASSO':Lasso(),
    'decision tree': DecisionTreeRegressor(),
    'random forest':RandomForestRegressor(),
    'extra trees': ExtraTreesRegressor(),
    'gradient boosting': GradientBoostingRegressor(),
    'adaboost': AdaBoostRegressor(),
    'mlp': MLPRegressor(),
    'xgboost':XGBRegressor()
}

In [87]:
model_output = []
for model_name,model in model_dict.items():
    model_output.append(scorer(model_name, model))

In [88]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])

In [89]:
model_df.sort_values(['mae'])

,name,r2,mae
5,random forest,0.901759,0.433637
6,extra trees,0.897685,0.447885
10,xgboost,0.899830,0.460699
7,gradient boosting,0.889207,0.500814
4,decision tree,0.817245,0.528294
9,mlp,0.842659,0.575785
8,adaboost,0.816971,0.695310
0,linear_reg,0.752754,0.760989
2,ridge,0.752783,0.761092
1,svr,0.691383,0.854937


### Hyperparameter Tuning

In [90]:
from sklearn.model_selection import GridSearchCV

In [91]:
param_grid = {
    'regressor__n_estimators': [50, 100, 200, 300],
    'regressor__max_depth': [None, 10, 20, 30],
    'regressor__max_samples':[0.1, 0.25, 0.5, 1.0],
    'regressor__max_features': ['auto', 'sqrt']
}

In [92]:
columns_to_encode = ['property_type','sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',handle_unknown='ignore',sparse_output=False),['agePossession']),
        ('target_enc', ce.TargetEncoder(), ['sector'])
    ], 
    remainder='passthrough'
)

In [93]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])

In [94]:
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

In [95]:
search = GridSearchCV(pipeline, param_grid, cv=kfold, scoring='r2', n_jobs=-1, verbose=4)

In [96]:
search.fit(X, y_transformed)

Fitting 10 folds for each of 128 candidates, totalling 1280 fits


,estimator,Pipeline(step...Regressor())])
,param_grid,"{'regressor__max_depth': [None, 10, ...], 'regressor__max_features': ['auto', 'sqrt'], 'regressor__max_samples': [0.1, 0.25, ...], 'regressor__n_estimators': [50, 100, ...]}"
,scoring,'r2'
,n_jobs,-1
,refit,True
,cv,KFold(n_split... shuffle=True)
,verbose,4
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,transformers,"[('num', ...), ('cat', ...), ...]"


In [97]:
final_pipe = search.best_estimator_

In [98]:
search.best_params_

{'regressor__max_depth': 20,
 'regressor__max_features': 'sqrt',
 'regressor__max_samples': 1.0,
 'regressor__n_estimators': 300}

In [99]:
search.best_score_

np.float64(0.9008983120049112)

In [100]:
final_pipe.fit(X,y_transformed)

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


### Exporting the model

In [101]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first',sparse_output=False),['sector','agePossession'])
    ], 
    remainder='passthrough'
)

In [102]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=500))
])

In [103]:
pipeline.fit(X,y_transformed)

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [104]:
import pickle

with open('pipeline.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

In [105]:
with open('df.pkl', 'wb') as file:
    pickle.dump(X, file)

In [106]:
X

,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 112,3,5,3+,Relatively New,3405.0,1,0,unfurnished,Low,Mid Floor
1,flat,sector 37c,3,3,3,Under Construction,1500.0,0,0,semifurnished,Medium,Low Floor
2,flat,sector 81,2,2,2,Relatively New,664.0,0,1,unfurnished,Low,Mid Floor
3,house,sector 43,5,5,2,Old Property,3323.0,1,0,semifurnished,Low,Mid Floor
4,flat,sector 92,4,4,3,Moderately Old,1893.0,0,0,semifurnished,Medium,Low Floor
...,...,...,...,...,...,...,...,...,...,...,...,...
3566,house,sector 31,4,4,2,Moderately Old,1557.0,1,0,unfurnished,Medium,Low Floor
3567,house,sector 28,4,5,3+,Relatively New,2844.0,1,1,unfurnished,Medium,Mid Floor
3568,flat,sector 107,3,2,2,Relatively New,750.0,0,0,semifurnished,Medium,Mid Floor
3569,flat,sohna road,2,2,2,Relatively New,1275.0,0,0,semifurnished,Low,Mid Floor


### Trying out the predictions

In [107]:
X.columns

Index(['property_type', 'sector', 'bedRoom', 'bathroom', 'balcony',
       'agePossession', 'built_up_area', 'servant room', 'store room',
       'furnishing_type', 'luxury_category', 'floor_category'],
      dtype='object')

In [108]:
X.iloc[0].values

array(['flat', 'sector 112', np.int64(3), np.int64(5), '3+',
       'Relatively New', np.float64(3405.0), np.int64(1), np.int64(0),
       'unfurnished', 'Low', 'Mid Floor'], dtype=object)

In [109]:
data = [['house', 'sector 102', 4, 3, '3+', 'New Property', 2750, 0, 0, 'unfurnished', 'Low', 'Low Floor']]
columns = ['property_type', 'sector', 'bedRoom', 'bathroom', 'balcony',
       'agePossession', 'built_up_area', 'servant room', 'store room',
       'furnishing_type', 'luxury_category', 'floor_category']

# Convert to DataFrame
one_df = pd.DataFrame(data, columns=columns)

one_df


,property_type,sector,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,house,sector 102,4,3,3+,New Property,2750,0,0,unfurnished,Low,Low Floor


In [110]:
np.expm1(pipeline.predict(one_df))

array([2.70550222])

In [111]:
X.dtypes

property_type       object
sector              object
bedRoom              int64
bathroom             int64
balcony             object
agePossession       object
built_up_area      float64
servant room         int64
store room           int64
furnishing_type     object
luxury_category     object
floor_category      object
dtype: object

In [112]:
sorted(X['sector'].unique().tolist())

['dwarka expressway',
 'gwal pahari',
 'manesar',
 'new',
 'new sector 2',
 'sector 1',
 'sector 102',
 'sector 103',
 'sector 104',
 'sector 105',
 'sector 106',
 'sector 107',
 'sector 108',
 'sector 109',
 'sector 10a',
 'sector 11',
 'sector 110',
 'sector 111',
 'sector 112',
 'sector 113',
 'sector 12',
 'sector 13',
 'sector 14',
 'sector 15',
 'sector 17',
 'sector 17a',
 'sector 17b',
 'sector 2',
 'sector 21',
 'sector 22',
 'sector 23',
 'sector 24',
 'sector 25',
 'sector 26',
 'sector 27',
 'sector 28',
 'sector 3',
 'sector 3 phase 2',
 'sector 3 phase 3 extension',
 'sector 30',
 'sector 31',
 'sector 33',
 'sector 36',
 'sector 36a',
 'sector 37',
 'sector 37c',
 'sector 37d',
 'sector 38',
 'sector 39',
 'sector 4',
 'sector 40',
 'sector 41',
 'sector 43',
 'sector 45',
 'sector 46',
 'sector 47',
 'sector 48',
 'sector 49',
 'sector 5',
 'sector 50',
 'sector 51',
 'sector 52',
 'sector 53',
 'sector 54',
 'sector 55',
 'sector 56',
 'sector 57',
 'sector 58',
 'sect